In [7]:
import pandas as pd
import requests
import json
import datetime

In [8]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [9]:
#req = requests.get('https://api.sncf.com/v1/coverage/sncf/disruptions//?since=20230201T000000&',auth=(token, ''))

In [10]:
req = requests.get('https://api.sncf.com/v1/coverage/sncf/disruptions?since=20230302T000000&start_page=0&count=1000&',auth=(token, ''))

In [11]:
doc = json.loads(req.text)
row = len(doc['disruptions'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 649


# Retard

## Liste des retards

In [12]:
df = pd.DataFrame(doc['disruptions'])
df.head(5)

,status,disruption_id,severity,impact_id,application_periods,updated_at,uri,impacted_objects,disruption_uri,contributor,cause,id,messages
0,future,77fd9129-ccd6-4d73-ba4a-2d48f7096675,"{'color': '#000000', 'priority': 42, 'name': '...",77fd9129-ccd6-4d73-ba4a-2d48f7096675,"[{'begin': '20230307T162500', 'end': '20230307...",20230302T121145,77fd9129-ccd6-4d73-ba4a-2d48f7096675,[{'impacted_stops': [{'amended_arrival_time': ...,77fd9129-ccd6-4d73-ba4a-2d48f7096675,realtime.sncf.piv,,77fd9129-ccd6-4d73-ba4a-2d48f7096675,NaN
1,future,71146c26-d919-40bd-8f22-318a70392114,"{'color': '#000000', 'priority': 42, 'name': '...",71146c26-d919-40bd-8f22-318a70392114,"[{'begin': '20230307T064000', 'end': '20230307...",20230302T121145,71146c26-d919-40bd-8f22-318a70392114,[{'impacted_stops': [{'amended_arrival_time': ...,71146c26-d919-40bd-8f22-318a70392114,realtime.sncf.piv,,71146c26-d919-40bd-8f22-318a70392114,NaN
2,future,145ac32e-9240-4857-b014-0af065222d35,"{'color': '#000000', 'priority': 42, 'name': '...",145ac32e-9240-4857-b014-0af065222d35,"[{'begin': '20230307T060000', 'end': '20230307...",20230302T121145,145ac32e-9240-4857-b014-0af065222d35,[{'impacted_stops': [{'amended_arrival_time': ...,145ac32e-9240-4857-b014-0af065222d35,realtime.sncf.piv,,145ac32e-9240-4857-b014-0af065222d35,NaN
3,future,598c10c3-786d-4f0e-9da2-f2545aa99ae7,"{'color': '#000000', 'priority': 42, 'name': '...",598c10c3-786d-4f0e-9da2-f2545aa99ae7,"[{'begin': '20230307T075000', 'end': '20230307...",20230302T121145,598c10c3-786d-4f0e-9da2-f2545aa99ae7,[{'impacted_stops': [{'amended_arrival_time': ...,598c10c3-786d-4f0e-9da2-f2545aa99ae7,realtime.sncf.piv,,598c10c3-786d-4f0e-9da2-f2545aa99ae7,NaN
4,past,b4b699aa-5b56-4079-97da-5a58f13acbcb,"{'color': '#000000', 'priority': 42, 'name': '...",b4b699aa-5b56-4079-97da-5a58f13acbcb,"[{'begin': '20230302T065700', 'end': '20230302...",20230302T121145,b4b699aa-5b56-4079-97da-5a58f13acbcb,[{'impacted_stops': [{'amended_arrival_time': ...,b4b699aa-5b56-4079-97da-5a58f13acbcb,realtime.sncf.piv,,b4b699aa-5b56-4079-97da-5a58f13acbcb,"[{'text': 'Agression d'un agent SNCF', 'channe..."


## Etat du retard

In [15]:
df_info = pd.DataFrame(list(df['severity']))
df_info.head(10)

,color,priority,name,effect
0,#000000,42,additional service,ADDITIONAL_SERVICE
1,#000000,42,additional service,ADDITIONAL_SERVICE
2,#000000,42,additional service,ADDITIONAL_SERVICE
3,#000000,42,additional service,ADDITIONAL_SERVICE
4,#000000,42,detour,DETOUR
5,#000000,42,trip delayed,SIGNIFICANT_DELAYS
6,#000000,42,trip delayed,SIGNIFICANT_DELAYS
7,#000000,42,trip delayed,SIGNIFICANT_DELAYS
8,#000000,42,trip delayed,SIGNIFICANT_DELAYS
9,#000000,42,additional service,ADDITIONAL_SERVICE


## Combinaison

In [16]:
df_fin = df_info.copy()
df_fin = df_fin[df_fin['effect'] == 'SIGNIFICANT_DELAYS']
df_fin = df_fin[['effect']]
df_fin

,effect
5,SIGNIFICANT_DELAYS
6,SIGNIFICANT_DELAYS
7,SIGNIFICANT_DELAYS
8,SIGNIFICANT_DELAYS
11,SIGNIFICANT_DELAYS
...,...
643,SIGNIFICANT_DELAYS
644,SIGNIFICANT_DELAYS
645,SIGNIFICANT_DELAYS
646,SIGNIFICANT_DELAYS


In [17]:
gare_d = []
gare_a = []
cause = []
retard = []
heure = []

for i in df_fin.index:
    df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]
    df_mod = df_mod[(df_mod['base_arrival_time'].notnull())]

    retard_val = int(df_mod.iloc[-1]['amended_arrival_time']) - int(df_mod.iloc[-1]['base_arrival_time'])

    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']

    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(df_mod.iloc[-1]['cause'])
    retard.append(str(retard_val)[:-2])
    heure.append(df_mod.iloc[0]['amended_departure_time'])

In [18]:
df_fin['gare_depart'] = gare_d
df_fin['gare_arrivee'] = gare_a
df_fin['cause'] = cause
df_fin['retard_min'] = retard
df_fin['heure_depart'] = heure

In [19]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_fin['heure_depart'] = df_fin['heure_depart'].apply(str_tps)

In [20]:
df_fin

,effect,gare_depart,gare_arrivee,cause,retard_min,heure_depart
5,SIGNIFICANT_DELAYS,Saint-Dizier,Paris Est,,5,05:33:00
6,SIGNIFICANT_DELAYS,Limoges-Bénédictins,Poitiers,Mise à quai tardive en gare origine,5,08:32:00
7,SIGNIFICANT_DELAYS,Abbeville,Amiens,Régulation du trafic,5,12:11:00
8,SIGNIFICANT_DELAYS,Besançon-Viotte,Dijon Ville,Modification de matériel,5,06:01:00
11,SIGNIFICANT_DELAYS,Sète,Avignon Centre,Indisponibilité d'un matériel,5,07:35:00
...,...,...,...,...,...,...
643,SIGNIFICANT_DELAYS,Metz Ville,Épinal,,5,11:59:00
644,SIGNIFICANT_DELAYS,Paris Nord,Amiens,Présence d'individus sur les voies,10,11:31:00
645,SIGNIFICANT_DELAYS,Paris - Montparnasse - Hall 1 & 2,Quimper,Mise à quai tardive en gare origine,60,13:04:30
646,SIGNIFICANT_DELAYS,Chalon-sur-Saône,Dijon Ville,Régulation du trafic,5,12:53:00
